In [2]:
import scanpy as sc
import pandas as pd
import decoupler as dc
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [3]:
# Define input and output files
if 'snakemake' in locals():
    adata_fp = snakemake.input[0]
    output_fp = snakemake.output[0]
    normalisation = snakemake.params[0]
    top_genes = snakemake.params[1]
else:
    adata_fp = '../../../data/merged.h5ad'
    output_fp = 'test.csv'
    normalisation = 'log1p'
    top_genes = 300

adata_spatial = sc.read_h5ad(adata_fp)

In [4]:
adata_spatial

AnnData object with n_obs × n_vars = 9129 × 5559
    obs: 'orig.ident', 'nCount_Spatial', 'nFeature_Spatial', 'Sample', 'plate', 'Confidence', 'PFI', 'patient', 'annotation_manual', 'B.cells_UCell', 'Cancer.cells_UCell', 'Dendritic.cells_UCell', 'Endothelial.cells_UCell', 'Fibroblasts_UCell', 'Meyloid.cells_UCell', 'T.cells_UCell', 'Ovarian.stromal.cells_UCell', 'Naturall.Killer.cells_UCell', 'Mast.cells_UCell', 'cell_type', 'seurat_clusters_old', 'PFI_confidence', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'in_tissue', 'array_row', 'array_col', '_indices', '_scvi_batch', '_scvi_labels'
    var: '_indices', 'features'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'mod', 'neighbors', 'spatial'
    obsm: 'X_harmony', 'X_pca', 'X_umap', 'means_cell_abundance_w_sf', 'mt', 'q05_cell_abundance_w_sf', 'q95_cell_abundance_w_sf', 'spatial', 'stds_cell_abundance_w_sf'
    varm: 'HARMONY', 'PCs'
    obsp: 'distances'

In [ ]:
if normalisation == 'log1p':
    sc.pp.normalize_total(adata_spatial, inplace=True)
    sc.pp.log1p(adata_spatial)
elif normalisation == 'SCT':
    adata_spatial.layers['counts'] = adata_spatial.X
    adata_spatial.X = adata_spatial.layers['SCT'].copy()
    del adata_spatial.layers['SCT']
    adata_spatial

In [ ]:
model = dc.get_progeny(organism='human', top=top_genes)
dc.run_mlm(mat=adata_spatial, net=model, source='source', target='target', weight='weight', verbose=True, use_raw=False)

In [ ]:
acts = dc.get_acts(adata_spatial, obsm_key='mlm_estimate')
print(acts)

In [ ]:
lims = pd.DataFrame({ 'llim' : [np.min(acts.X[:,ii]) for ii in range(acts.n_vars)], 'ulim': [np.max(acts.X[:,ii]) for ii in range(acts.n_vars)]}, index = acts.var_names.values)
lims['lim'] = [np.max(abs(acts.X[:,ii])) for ii in range(acts.n_vars)]
print('Max and min values per pathway')
print(lims)